# Natural language processing
## Bert to Tensorflow Serving

## 1 - Dependencies

In [ ]:
!pip install transformers --quiet

In [ ]:
# Donwload pretrained BERT for classification
import os
import gdown
if os.access('bert_weights.h5', os.F_OK) is False:
    url = 'https://drive.google.com/uc?id=1ZZaAKr4jb9eLSora5kbSg8dJqb4DkjzL&export=download'
    output = 'bert_weights.h5'
    gdown.download(url, output, quiet=False)
else:
    print("Los pesos bert_weights.h5 ya están descargados")

In [ ]:
# Load model
import tensorflow as tf
import numpy as np
from transformers import BertTokenizer, TFBertModel, BertConfig

output_shape = 3
max_length = 140

bert_model = TFBertModel.from_pretrained("bert-base-uncased")

input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='input_ids')
attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='attention_mask')

# Get the pooled_output (embedding que representa toda la entrada)
output = bert_model.bert(input_ids=input_ids, attention_mask=attention_mask)[1]

# We can also add dropout as regularization technique:
output = tf.keras.layers.Dropout(rate=0.2)(output)

# Se puede agregar más capas Densas en el medio si se desea

# Provide number of classes to the final layer:
output = tf.keras.layers.Dense(output_shape, activation='softmax')(output)

# Final model:
model = tf.keras.models.Model(inputs=[input_ids, attention_mask], outputs=output)
model.load_weights("bert_weights.h5")

In [ ]:
# Export to TFX model
callable = tf.function(model.call)
concrete_function = callable.get_concrete_function([tf.TensorSpec((None, max_length), tf.int32, name="input_ids"), tf.TensorSpec((None, max_length), tf.int32, name="attention_mask")])
model.save('mybert/1', signatures=concrete_function)

In [ ]:
# Zip model for download
!zip -r mybert.zip mybert

In [ ]:
# Connect to google drive to copy and download the zip model
import shutil
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
print(shutil.copyfile('mybert.zip', os.path.join("/content/drive/MyDrive", 'mybert.zip')))